# DP Aggregate Seeded Synthesizer
DP Aggregate Seeded synthesizer is a differentially private synthesizer that relies on DP Marginals to build synthetic data. It will compute DP Marginals (called aggregates) for your dataset using an specified `reporting length`, and synthesize data based on the computed aggregated counts.

> Aggregates will be computed for all lengths of attribute combination up to and including the `reporting length`.

## 1. Overview

### 1.1. Aggregate data generation with DP
Let's consider the following input as example:

| A | B | C |
| -- | -- | -- |
| a1 | b1 | c1 |
| a1 | b2 | c1 |
| a2 |    | c2 |
| a2 | b2 | c1 |
| a1 | b2 |    |

The input data is assumed to be categorical and the domain will be inferred from the input dataset:

- `A` possible values are `a1,a2`
- `B` possible values are `b1,b2`
- `C` possible values are `c1,c2`

For a `reporting length=2`, the aggregates in the dataset above could be:

- 1-counts
    - `A:a1`: 3 + NOISE
    - `A:a2`: 2 + NOISE
    - `B:b1`: 1 + NOISE
    - `B:b2`: 3 + NOISE
    - `C:c1`: 3 + NOISE
    - `C:c2`: 1 + NOISE

- 2-counts:
    - `A:a1;C:c1`: 2 + NOISE
    - `A:a2;B:b2`: 1 + NOISE
    - `B:b1;C:c1`: 1 + NOISE
    - `A:a1;B:b1`: 1 + NOISE
    - `A:a1;B:b2`: 2 + NOISE
    - `B:b2;C:c1`: 2 + NOISE
    - `A:a2;C:c2`: 1 + NOISE
    - `B:b2;C:c2`: 0 + NOISE

Also, some spurious combinations might be created and reported to ensure differentially private guarantees - notice that `B:b2;C:c2` does not exist in the sensitive dataset, but it has been _fabricated_ and added to the output.

Similarly, some attribute combinations might be suppressed. For example, even though `A:a2;C:c1` exists in the sensitive dataset, it has not been reported as an aggregate.

### 1.2. Synthesis
Data will be then synthesized directly from the aggregates computed with differential privacy to produce synthetic data. Which will ensure the same DP guarantees to the synthetic data.

## 2. Imports and global config

In [22]:
import pandas as pd

from sdssynth import init_logger, set_number_of_threads
from sdssynth import Dataset
from sdssynth import DpAggregateSeededParametersBuilder, AccuracyMode, FabricationMode
from sdssynth import DpAggregatedSeededSynthesizer, Dataset

from utils import gen_dataset, ErrorReport

# The library allows the desired log level to be set if wanted
# ('off' || 'error' || 'warn' || 'info' || 'debug' || 'trace')
# init_logger('trace')

# some algorithms have parallel implementations, so the desired number of threads can be set
# (the default is one thread per CPU core)
# set_number_of_threads(2)

## 3. Generating an example data frame with random data

> `gen_dataset` is just an utility to generate some example data (the code for it is in [`utils.py`](./utils.py))

To illustrate the library, let's start by creating an example data frame:

In [23]:
number_of_records_to_generate = 6000

sensitive_df = gen_dataset(number_of_records_to_generate)

sensitive_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,,1,3,0,1,1,1,1,1,0
1,,,1,1,0,1,0,1,1,1
2,1,3,4,0,0,0,1,1,0,1
3,2,2,4,0,1,0,0,1,0,1
4,1,,3,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
5995,1,4,9,0,1,0,0,1,1,0
5996,2,4,6,1,0,1,0,1,0,1
5997,,5,,1,1,0,0,1,0,1
5998,1,4,9,0,0,0,1,1,1,0


## 4. Creating the sensitive dataset

### 4.1. Creation from constructor

The library uses an internal representation of the data to optimize execution time complexity.

If the data is already in the required raw format, you can call the constructor directly:

In [24]:
sensitive_raw_data = [
    # headers
    ['A', 'B', 'C', 'D'],
    # records
    ['a1', 'b1', 'c1', '0'],
    ['a1', 'b1', 'c2', '0'],
    ['a2', '', 'c2', '1'],
]
sensitive_dataset = Dataset(sensitive_raw_data)

### 4.2. Negative value interpretation

The library distinguishes 'positive' attribute values that indicate the presence of specific sensitive data from 'negative' attribute values that indicate the absence of such data. By default, the integer zero (`0`) and the empty string (`""`) are not taken into account when creating and counting attribute combinations. Any columns where zero values are of interest (and thus sensitive) should be listed as `sensitive_zeros`, so they will be treated the same way as positive values.

> For more parameters see the library documentation.

In [25]:
sensitive_raw_data = [
    # headers
    ['A', 'B', 'C', 'D'],
    # records
    ['a1', 'b1', 'c1', '0'],
    ['a1', 'b1', 'c2', '0'],
    ['a2', '', 'c2', '1'],
]
sensitive_dataset = Dataset(sensitive_raw_data, sensitive_zeros=['D'])

### 4.3. Creating sensitive dataset from a pandas data frame
For convenience, a method is provided to build a dataset from a pandas data frame:

In [26]:
sensitive_dataset = Dataset.from_data_frame(sensitive_df)

## 5. Generating the synthetic data

### 5.1. Defining synthesizer parameters

If you just want to create the synthesizer with default parameters:

In [27]:
synth = DpAggregatedSeededSynthesizer()
print (synth.parameters)

{
  "reporting_length": 3,
  "epsilon": 0.1,
  "delta": null,
  "percentile_percentage": 99,
  "percentile_epsilon_proportion": 0.01,
  "sigma_proportions": [
    1.0,
    0.5,
    0.3333333333333333
  ],
  "number_of_records_epsilon": 0.1,
  "threshold": {
    "type": "Adaptive",
    "valuesByLen": {
      "3": 1.0,
      "2": 1.0
    }
  },
  "empty_value": "",
  "use_synthetic_counts": false,
  "weight_selection_percentile": 95,
  "aggregate_counts_scale_factor": null
}


However, this might not produce the optimal output for your dataset and downstream analysis tasks.

To tune the synthesizer parameters, we provide a builder (`DpAggregateSeededParametersBuilder`). Any parameters passed to the builder will override the default values accordingly:

In [28]:
# this explicitly outlines the default parameters
synth = DpAggregateSeededParametersBuilder() \
        .reporting_length(3) \
        .epsilon(6.0) \
        .delta(1 / (2.0 * len(sensitive_df))) \
        .percentile_percentage(99) \
        .percentile_epsilon_proportion(0.01) \
        .accuracy_mode(AccuracyMode.prioritize_large_counts()) \
        .number_of_records_epsilon(0.1) \
        .fabrication_mode(FabricationMode.uncontrolled()) \
        .empty_value("") \
        .weight_selection_percentile(95) \
        .use_synthetic_counts(False) \
        .aggregate_counts_scale_factor(1.0) \
        .build()
print (synth)


{
  "reporting_length": 3,
  "epsilon": 6.0,
  "delta": 0.00008333333333333333,
  "percentile_percentage": 99,
  "percentile_epsilon_proportion": 0.01,
  "sigma_proportions": [
    1.0,
    0.5,
    0.3333333333333333
  ],
  "number_of_records_epsilon": 0.1,
  "threshold": {
    "type": "Adaptive",
    "valuesByLen": {
      "3": 1.0,
      "2": 1.0
    }
  },
  "empty_value": "",
  "use_synthetic_counts": false,
  "weight_selection_percentile": 95,
  "aggregate_counts_scale_factor": 1.0
}


To continue with this example, let's set the parameters we care about for now:

In [29]:
reporting_length = 4

builder = DpAggregateSeededParametersBuilder() \
    .reporting_length(reporting_length) \
    .epsilon(0.2) \
    .accuracy_mode(AccuracyMode.prioritize_large_counts()) \
    .fabrication_mode(FabricationMode.uncontrolled())

synth = DpAggregatedSeededSynthesizer(builder.build())

### 5.2 Building the model and synthesizing data

In [30]:
synth.fit(sensitive_dataset)

# here if we do not specify the desired number of samples, the synthesizer will
# use all the available attributes based on the 1-counts to synthesize records
synthetic_raw_data = synth.sample(len(sensitive_df))
synthetic_dataset = Dataset(synthetic_raw_data)

# as an example, let's create a pandas data frame from the raw synthetic data
synthetic_df = Dataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,,,1,1,1,1,1,1,1
1,1,,,1,1,1,1,1,1,1
2,1,,,1,1,1,1,1,1,1
3,1,,,1,1,1,1,1,1,1
4,1,,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5995,,6,1,,,,,,,
5996,,6,1,,,,,,,
5997,,6,1,,,,,,,
5998,,6,3,,,,,,,


## 6. Generating/exporting aggregate data

In [31]:
# generate sensitive aggregates
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ';')

# export the differentially private aggregates (internal to the synthesizer)
dp_aggregates = synth.get_dp_aggregates(';')

# generate aggregates from the synthetic data
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ';')

# let's take a look at the DP aggregates
list(dp_aggregates.items())[:20]

[('H2:6;H3:10;H5:1;H7:1', 12),
 ('H1:1;H3:6;H4:1;H8:1', 75),
 ('H1:1;H3:7;H6:1', 7),
 ('H2:6;H3:9;H8:1', 82),
 ('H10:1;H3:6;H8:1', 176),
 ('H1:2;H2:5;H4:1', 137),
 ('H2:1;H3:2;H6:1;H8:1', 30),
 ('H10:1;H1:2;H5:1;H6:1', 302),
 ('H10:1;H3:7;H5:1', 184),
 ('H3:1;H4:1;H7:1;H8:1', 81),
 ('H2:2;H3:10;H4:1', 9),
 ('H2:2;H3:2;H5:1;H8:1', 30),
 ('H3:10;H4:1;H6:1;H7:1', 128),
 ('H10:1;H1:1;H6:1;H8:1', 203),
 ('H1:1;H3:8;H4:1', 215),
 ('H1:1;H7:1;H8:1', 440),
 ('H1:2;H3:10;H5:1;H9:1', 14),
 ('H1:2;H2:4;H3:5', 12),
 ('H1:2;H5:1;H6:1;H9:1', 232),
 ('H2:2;H3:4;H4:1;H7:1', 66)]

## 7. Evaluating DP aggregates and DP synthetic data

This section is an example evaluation of both the DP aggregates and the DP synthetic data, as well as the influence of some synthesizer parameters in the DP aggregates and synthetic data.

### 7.1. Evaluating current results

> `ErrorReport` is just an example way to evaluate results (the code for it is [`utils.py`](./utils.py))

- **Count**: mean of the aggregate counts for the given length
- **Error**: mean of the `abs(sensitive_count - dp_aggregated_count)` or `abs(sensitive_count - synthetic_count)`
- **Suppressed %**: percentage of combinations present in the sensitive dataset, but not present in the aggregated/synthetic data
- **Fabricated %**: percentage of combinations that were reported in the aggregated/synthetic data, but do not exist in the sensitive dataset

In [32]:
sensitive_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive Data vs. DP Aggregates**

In [33]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1373.84 +/- 46.64,0.00 %,0.00 %
1,2,422.07 +/- 45.87,0.96 %,7.59 %
2,3,147.71 +/- 43.25,10.69 %,9.19 %
3,4,55.85 +/- 27.68,40.85 %,4.85 %
4,All,110.50 +/- 34.55,30.44 %,6.51 %


**Sensitive Data vs. DP Synthetic Data**

In [34]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1373.84 +/- 92.00,0.00 %,0.00 %
1,2,422.07 +/- 129.22,0.96 %,7.17 %
2,3,147.71 +/- 82.11,10.92 %,8.29 %
3,4,55.85 +/- 52.34,41.94 %,4.02 %
4,All,110.50 +/- 69.22,31.24 %,5.71 %


### 7.2. Targeting less fabrication

Let's update the current synthesizer parameters to `minimize` fabrication:

In [35]:
synth = DpAggregatedSeededSynthesizer(
    builder \
        .fabrication_mode(FabricationMode.minimize()) \
        .build()
)

synth.fit(sensitive_dataset)

synthetic_raw_data = synth.sample(len(sensitive_df))
synthetic_dataset = Dataset(synthetic_raw_data)

synthetic_df = Dataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,,,1,1,1,1,1,1,1
1,1,,,1,1,1,1,1,1,1
2,1,,,1,1,1,1,1,1,1
3,1,,,1,1,1,1,1,1,1
4,1,,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5995,2,6,,,,,,,,
5996,2,6,,,,,,,,
5997,2,6,,,,,,,,
5998,2,6,,,,,,,,


Evaluating again:

In [36]:
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ';')
dp_aggregates = synth.get_dp_aggregates(';')
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ';')

sensitive_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive Data vs. DP Aggregates**

In [37]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1373.84 +/- 60.08,0.00 %,0.00 %
1,2,422.07 +/- 55.74,22.01 %,0.00 %
2,3,147.71 +/- 47.29,73.65 %,0.00 %
3,4,55.85 +/- 38.32,92.88 %,0.00 %
4,All,110.50 +/- 47.58,83.08 %,0.00 %


**Sensitive Data vs. DP Synthetic Data**

In [38]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1373.84 +/- 255.96,0.00 %,0.00 %
1,2,422.07 +/- 255.87,22.01 %,0.00 %
2,3,147.71 +/- 339.16,73.65 %,0.00 %
3,4,55.85 +/- 619.93,92.88 %,0.00 %
4,All,110.50 +/- 392.89,83.08 %,0.00 %


### 7.3. Prioritize small counts

Let's update the current synthesizer parameters to `prioritize_small_counts`:

In [39]:
synth = DpAggregatedSeededSynthesizer(
    builder \
        .fabrication_mode(FabricationMode.uncontrolled()) \
        .accuracy_mode(AccuracyMode.prioritize_small_counts()) \
        .use_synthetic_counts(False) \
        .build()
)

synth.fit(sensitive_dataset)

synthetic_raw_data = synth.sample(len(sensitive_df))
synthetic_dataset = Dataset(synthetic_raw_data)

synthetic_df = Dataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,,,,1,1,1,1,1,1,1
1,,,,1,1,1,1,1,1,1
2,,,,1,1,1,1,1,1,1
3,,,,1,1,1,1,1,1,1
4,,,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5995,2,3,3,,,,,,,
5996,2,3,3,,,,,,,
5997,2,3,3,,,,,,,
5998,2,3,7,,,,,,,


Evaluating again:

In [40]:
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ';')
dp_aggregates = synth.get_dp_aggregates(';')
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ';')

sensitive_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive Data vs. DP Aggregates**

In [41]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1373.84 +/- 11.60,0.00 %,0.00 %
1,2,422.07 +/- 37.80,0.00 %,7.52 %
2,3,147.71 +/- 55.13,16.10 %,8.86 %
3,4,55.85 +/- 41.32,63.26 %,5.80 %
4,All,110.50 +/- 46.14,46.94 %,7.19 %


**Sensitive Data vs. DP Synthetic Data**

In [42]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1373.84 +/- 206.08,0.00 %,0.00 %
1,2,422.07 +/- 147.19,0.00 %,7.52 %
2,3,147.71 +/- 84.32,16.76 %,7.59 %
3,4,55.85 +/- 48.46,64.68 %,4.85 %
4,All,110.50 +/- 76.65,48.07 %,6.24 %
